In [2]:
%loadFromPOM pom.xml

In [3]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import org.hl7.fhir.r4.model.*;

In [4]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

背景：阿里史診所是一間位於台灣東部的小診所，地址為宜蘭縣三星鄉拱照八路86號，診所只有醫師兩名輪流看診。分別是陳莎莎醫師和陳柴柴醫師，都是全科的醫師。

產生基本診所資料

In [ ]:
// Add Location Data 阿里史診所資料
Location local = new Location();
local.setName("阿里史診所");

Address address = new Address();
address.addLine("宜蘭縣三星鄉拱照八路86號");
address.setCity("宜蘭縣");
address.setDistrict("三星鄉");
address.setPostalCode("266");
address.setCountry("台灣");
local.setAddress(address);

ContactPoint contact = new ContactPoint();
contact.setSystem(ContactPoint.ContactPointSystem.PHONE);
contact.setValue("03-989-1234");
local.addTelecom(contact);

// print Location Data
String orgString = ctx.newJsonParser().setPrettyPrint(true).encodeResourceToString(local);
System.out.println(orgString);

產生醫師資料

In [ ]:
// New Practitioner Identifier
Identifier identifier = new Identifier();
identifier.setUse(Identifier.IdentifierUse.OFFICIAL);
identifier.setType(new CodeableConcept().addCoding(new Coding().setSystem("http://terminology.hl7.org/CodeSystem/v2-0203").setCode("MD").setDisplay("醫師")));

// new Qualification 附加資格
Practitioner.PractitionerQualificationComponent qualification = new Practitioner.PractitionerQualificationComponent();
qualification.setCode(new CodeableConcept().addCoding(new Coding().setSystem("https://twcore.mohw.gov.tw/ig/twcore/CodeSystem/health-professional").setCode("A")));

// Add Practitioner 1 陳莎莎醫師資料，with qualification 附加資格
Practitioner practitioner1 = new Practitioner();
practitioner1.addName()
    .setFamily("陳")
    .addGiven("莎莎");
practitioner1.addTelecom()
    .setSystem(ContactPoint.ContactPointSystem.PHONE)
    .setValue("03-12345678");
practitioner1.addTelecom()
    .setSystem(ContactPoint.ContactPointSystem.PHONE)
    .setValue("0987-666471");
practitioner1.addIdentifier(identifier);
practitioner1.addQualification(qualification);

// add practitioner 2 陳柴柴醫師資料，with qualification 附加資格
Practitioner practitioner2 = new Practitioner();
practitioner2.addName()
    .setFamily("陳")
    .addGiven("柴柴");
practitioner2.addTelecom()
    .setSystem(ContactPoint.ContactPointSystem.PHONE)
    .setValue("03-12345678");
practitioner2.addTelecom()  
    .setSystem(ContactPoint.ContactPointSystem.PHONE)
    .setValue("0987-666472");
practitioner2.addIdentifier(identifier);
practitioner2.addQualification(qualification);


以Bundle格式處理

In [ ]:
// create bundle for all data
Bundle bundle = new Bundle();
bundle.setType(Bundle.BundleType.TRANSACTION);
bundle.addEntry()
    .setFullUrl(local.getIdElement().getValue())
    .setResource(local)
    .getRequest()
    .setUrl("Location")
    .setMethod(Bundle.HTTPVerb.POST);
    bundle.addEntry()
    .setFullUrl(practitioner1.getIdElement().getValue())
    .setResource(practitioner1)
    .getRequest()
    .setUrl("Practitioner")
    .setMethod(Bundle.HTTPVerb.POST);
    bundle.addEntry()
    .setFullUrl(practitioner2.getIdElement().getValue())
    .setResource(practitioner2)
    .getRequest()
    .setUrl("Practitioner")
    .setMethod(Bundle.HTTPVerb.POST);

// save data to server - Run one time only
// Bundle resp = client.transaction().withBundle(bundle).execute();
// System.out.println(ctx.newJsonParser().setPrettyPrint(true).encodeResourceToString(resp));

產生Practitioner Role資料

In [ ]:
//read location data "阿里史診所"
Bundle bundle1 = client
    .search()
    .forResource(Location.class)
    .where(Location.NAME.matches().value("阿里史診所"))
    .returnBundle(Bundle.class)
    .execute();
Location location = (Location) bundle1.getEntry().get(0).getResource();

// read practitioner data "陳莎莎醫師"
Bundle bundle2 = client
    .search()
    .forResource(Practitioner.class)
    .where(Practitioner.NAME.matches().value("莎莎"))
    .returnBundle(Bundle.class)
    .execute();
Practitioner practitioner = (Practitioner) bundle2.getEntry().get(0).getResource();

PractitionerRole practitionerRole1 = new PractitionerRole();
practitionerRole1.addLocation(new Reference("Location/" + location.getIdElement().getIdPart()));
practitionerRole1.setPractitioner(new Reference("Practitioner/" + practitioner.getIdElement().getIdPart()));
// add specialty
CodeableConcept specialty = new CodeableConcept();
specialty.addCoding(new Coding().setCode("394814009").setDisplay("家庭醫學科").setSystem("http://snomed.info/sct"));
practitionerRole1.addSpecialty(specialty);

// read practitioner data "陳柴柴醫師"
Bundle bundle3 = client
    .search()
    .forResource(Practitioner.class)
    .where(Practitioner.NAME.matches().value("柴柴"))
    .returnBundle(Bundle.class)
    .execute();
Practitioner practitioner2 = (Practitioner) bundle3.getEntry().get(0).getResource();

PractitionerRole practitionerRole2 = new PractitionerRole();
practitionerRole2.addLocation(new Reference("Location/" + location.getIdElement().getIdPart()));
practitionerRole2.setPractitioner(new Reference("Practitioner/" + practitioner2.getIdElement().getIdPart()));
// add specialty
CodeableConcept specialty2 = new CodeableConcept();
specialty2.addCoding(new Coding().setCode("394814009").setDisplay("家庭醫學科").setSystem("http://snomed.info/sct"));
practitionerRole2.addSpecialty(specialty2);

// create bundle for all data
Bundle bundle4 = new Bundle();
bundle4.setType(Bundle.BundleType.TRANSACTION);
bundle4.addEntry()
    .setFullUrl(practitionerRole1.getIdElement().getValue())
    .setResource(practitionerRole1)
    .getRequest()
    .setUrl("PractitionerRole")
    .setMethod(Bundle.HTTPVerb.POST);
bundle4.addEntry()
    .setFullUrl(practitionerRole2.getIdElement().getValue())
    .setResource(practitionerRole2)
    .getRequest()
    .setUrl("PractitionerRole")
    .setMethod(Bundle.HTTPVerb.POST);

// save data to server - Run one time only
//Bundle resp = client.transaction().withBundle(bundle4).execute();
//System.out.println(ctx.newJsonParser().setPrettyPrint(true).encodeResourceToString(resp));

In [10]:
// 阿里史診所 隸屬於 東部醫療網路 宜蘭縣宜蘭市中山路2號 電話：03-989-4321   
Organization organization = new Organization();
organization.setName("東部醫療網路");

Address address = new Address();
address.addLine("宜蘭縣宜蘭市中山路2號");
address.setCity("宜蘭縣");
address.setDistrict("宜蘭市");
address.setPostalCode("260");
address.setCountry("台灣");
organization.addAddress(address);

ContactPoint contact = new ContactPoint();
contact.setSystem(ContactPoint.ContactPointSystem.PHONE);
contact.setValue("03-989-4321");
organization.addTelecom(contact);

// save data to server - Run one time only
//MethodOutcome outcome = client.create().resource(organization).execute();
//System.out.println(outcome.getId().getValue());

// read organization data "東部醫療網路"
Bundle bundle5 = client
    .search()
    .forResource(Organization.class)
    .where(Organization.NAME.matches().value("東部醫療網路"))
    .returnBundle(Bundle.class)
    .execute();
Organization organization = (Organization) bundle5.getEntry().get(0).getResource();

// read location data "阿里史診所"
Bundle bundle6 = client
    .search()
    .forResource(Location.class)
    .where(Location.NAME.matches().value("阿里史診所"))
    .returnBundle(Bundle.class)
    .execute();
Location location = (Location) bundle6.getEntry().get(0).getResource();

// add organization to location - run one time only
// location.setManagingOrganization(new Reference("Organization/" + organization.getIdElement().getIdPart()));


org.hl7.fhir.r4.model.Location@135c6be8

In [ ]:
//read Organization "東部醫療網路" from DB
Bundle response = client
                .search()
                .forResource(Organization.class)
                .where(Organization.NAME.matchesExactly().value("東部醫療網路"))
                .returnBundle(Bundle.class)
                .execute();
Organization organization = (Organization) response.getEntry().get(0).getResource();

// Read all patient from DB
Bundle response = client.search().forResource(Patient.class).returnBundle(Bundle.class).execute();

// count patients
int count = 0;
while (true) {
    for (Bundle.BundleEntryComponent entry : response.getEntry()) {
        Resource resource = entry.getResource();
        if (resource instanceof Patient) {
            Patient patient = (Patient) resource;
            // Set general practitioner as "東部醫療網路"
            patient.setGeneralPractitioner(List.of(new Reference("Organization" + "/" + organization.getIdElement().getIdPart())));
            // Update patient
            MethodOutcome outcome = client.update().resource(patient).execute();
            //Print patient name & patient id
            System.out.println(patient.getName().get(0).getGivenAsSingleString() + " " + patient.getName().get(0).getFamily() + " " + patient.getId());
            count++;
        }
    }
    // Load next page
    if (response.getLink(Bundle.LINK_NEXT) != null) {
        response = client.loadPage().next(response).execute();
    } else {
        break;
    }
}
System.out.println("Total patients: " + count);